In [1]:
import boto3

In [2]:
import configparser

In [3]:
config = configparser.ConfigParser()

In [4]:
config.read('aws.cfg')

['aws.cfg']

In [5]:
AWS_ACCESS_KEY = config['AWS']['aws_access_key']
AWS_SECRET_KEY = config['AWS']['aws_secret_key']

In [6]:
#s3client = boto3.client('s3', aws_access_key_id = AWS_ACCESS_KEY, aws_secret_access_key = AWS_SECRET_KEY)

In [7]:
#response = s3client.list_buckets()

In [8]:
bucket_name = 'techcatalyst-raw'

In [9]:
# for bucket in response['Bucket']:
#     print(bucket['Name'])

In [12]:
# obj_r = s3client.list_objects_vs(Bucket=bucket_name)
# for obj in obj_r['Contents']:
#     print(obj['Key'])
# uri = f's3://{bucket_name}/{myfiles[0]}'

import pandas as pd
# df = pd.read_parquet(uri, storage_options= {'key': AWS_ACCESS_KEY, 'secret': AWS_SECRET_KEY})

In [13]:
#Make method with all data transformations and then loop through files to process efficiently

In [17]:
from datetime import datetime

stats = pd.DataFrame()
def cleanup(df, name, dest):
    #Read Data
    # df = pd.read_parquet('yellow_tripdata_2024-04.parquet')
    df = pd.read_parquet(name)
    
    df['Trip_Duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() // 60
    df['Total_Trip_Charge'] = df['fare_amount'] + df['extra'] + df['mta_tax'] + df['tolls_amount'] + df['improvement_surcharge'] + df['congestion_surcharge'] + df['Airport_fee'] + df['tip_amount']
    df['Trip_Date'] = pd.to_datetime(df['tpep_pickup_datetime'].dt.date)
    df['Trip_Month'] = df['Trip_Date'].dt.month_name()
    df['Trip_Day'] = df['Trip_Date'].dt.day_name()
    df['Trip_Year'] = df['Trip_Date'].dt.year
    cols = ['VendorID', 'Trip_Date', 'Trip_Year', 'Trip_Month', 'Trip_Day',
       'passenger_count', 'trip_distance', 'store_and_fwd_flag',
       'payment_type', 'Trip_Duration',
       'Total_Trip_Charge' ]
    df = df[cols]
    df.rename(columns={
    'VendorID': 'Vendor_ID',
    'passenger_count': 'No_of_Passengers',
    'store_and_fwd_flag': 'SF_Flag',
    'payment_type': 'Payment_Type'
}, inplace=True)
    
    #Write Data
    # df.to_parquet('Yellow Taxi Transformed.parquet', partition_cols=['Trip_Year', 'Trip_Month'])
    df.to_parquet(dest, partition_cols=['Trip_Year', 'Trip_Month'])
    stats_data = {'file_name': name ,'number of records': df.shape[0] ,'number_of_cols': df.shape[1] ,'number_of_cols_with_na': df.isna().any().sum(),'date_time': datetime.now()} 
    stats_return = pd.DataFrame(data=stats_data, index=[0])
    return stats_return
stats = pd.DataFrame()
obj = ['yellow_tripdata_2024-04.parquet', 'yellow_tripdata_2024-03.parquet']
for ob in obj:
    test = cleanup(pd.DataFrame(), ob, 'Yellow Taxi Transformed.parquet')
    stats = pd.concat([stats, test], axis=0)
stats


,file_name,number of records,number_of_cols,number_of_cols_with_na,date_time
0,yellow_tripdata_2024-04.parquet,3514289,11,3,2024-06-27 10:55:28.472559
0,yellow_tripdata_2024-03.parquet,3582628,11,3,2024-06-27 10:55:35.948348
